# lab 09 -1
이하 모든 코드는 tf 2.0-beta 버전에서 실행가능하도록 수정함

In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
#tf.enable_eager_execution() - default in tf  2.0
tf.random.set_seed(0)

## TF session

In [24]:

dataset=tf.data.Dataset.from_tensor_slices((x_data,y_data)).shuffle(50000).batch(len(x_data)) # shuffle --> 이후 iterator 없음
def preproc_data(features,labels):
    features=tf.cast(features,tf.float32)
    labels=tf.cast(labels,tf.float32)
    return features, labels

In [26]:
##weight&bias 생성 
#random_normal 2.0에서 사용불가
W1 = tf.Variable(tf.random.normal([2, 1]), name='weight1')
b1 = tf.Variable(tf.random.normal([1]), name='bias1')

W2 = tf.Variable(tf.random.normal([2, 1]), name='weight2')
b2 = tf.Variable(tf.random.normal([1]), name='bias2')

W3 = tf.Variable(tf.random.normal([2, 1]), name='weight3')
b3 = tf.Variable(tf.random.normal([1]), name='bias3')

In [29]:
#layer 생성
def neural_net(features):
    layer1 = tf.sigmoid(tf.matmul(features, W1) + b1)  ## .nn 없이 사용가능? 가능! ##softmax는 사용불가 nn. or math.필요
    layer2 = tf.sigmoid(tf.matmul(features, W2) + b2)
    layer3 = tf.concat([layer1, layer2],-1) ## axis=-1
    layer3 = tf.reshape(layer3, shape = [-1,2])
    hypothesis = tf.sigmoid(tf.matmul(layer3, W3) + b3)
    return hypothesis
#loss func(in logistic reg) 
#.math.log로 수정할 것
def loss_fn(hypothesis, labels):
    cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + (1 - labels) * tf.math.log(1 - hypothesis))
    return cost

#optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
optimizer=tf.keras.optimizers.SGD(learning_rate=0.01)


def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32) ##boolean 값을 float32값으로 cast
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32)) #실제 y값과 예측값의 비교
    return accuracy

def grad(hypothesis, features, labels):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(neural_net(features),labels)
    return tape.gradient(loss_value, [W1, W2, W3, b1, b2, b3])
#keras - loss func & accuracy 이용
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

In [31]:
EPOCHS = 50000

for step in range(EPOCHS):
    for features, labels  in dataset:
  #  for features, labels  in tf.Iterator(dataset):--> shuffle로 인해 불필요
        features, labels = preproc_data(features, labels)
        grads = grad(neural_net(features), features, labels)
        optimizer.apply_gradients(grads_and_vars=zip(grads,[W1, W2, W3, b1, b2, b3]))
        if step % 5000 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(neural_net(features),labels)))
x_data, y_data = preproc_data(x_data, y_data)
test_acc = accuracy_fn(neural_net(x_data),y_data)
print("Testset Accuracy: {:.4f}".format(test_acc))

Iter: 0, Loss: 0.1466
Iter: 5000, Loss: 0.0925
Iter: 10000, Loss: 0.0656
Iter: 15000, Loss: 0.0503
Iter: 20000, Loss: 0.0406
Iter: 25000, Loss: 0.0339
Iter: 30000, Loss: 0.0291
Iter: 35000, Loss: 0.0254
Iter: 40000, Loss: 0.0226
Iter: 45000, Loss: 0.0203
Testset Accuracy: 1.0000


#### 삭제
for i in range(300):
  grads = grad(model, training_inputs, training_outputs)
  optimizer.apply_gradients(zip(grads, [model.W, model.B]))
  if i % 20 == 0:
    print("Loss at step {:03d}: {:.3f}".format(i, loss(model, training_inputs, training_outputs)))

print("Final loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))
print("W = {}, B = {}".format(model.W.numpy(), model.B.numpy()))

@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)